In [ ]:
# Get config
from config import user
from config import password

In [ ]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [ ]:
# Create consumer

# Imports
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json

#KAFKA variables, Move to the OS variables or configuration
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "group5cgm"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe([confluentTopicName])

In [ ]:
# Database variables
database = "group5database"
table = "dbo.CGM_Stream"
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"
port = "1433"

In [ ]:
aString = {}

kafkaListDictionaries = []

for i in range(576):
    try:
        
        # Collect message
        msg = c.poll(timeout=30)
        print(msg)
        
        # Error messages
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            
            # Get string
            datum = json.loads('{}'.format(msg.value().decode('utf-8')))
            #datum['timestamp'] = msg.timestamp()[1]
            
            # Convert to DataFrame
            for x in datum:
                df = spark.createDataFrame([x])
                # Add to table
                df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
                    .mode("append") \
                    .option("dbtable", table) \
                    .option("user", user) \
                    .option("password", password) \
                    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
                    .save()
            
                print(f'Patient {x["PtID"]}: Blood glucose level of {x["Glucose"]}')
            
    except Exception as e:
        print(e)